<a href="https://colab.research.google.com/github/KoNGoL/tfm_breast_cancer_detection/blob/main/Retinanet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!mkdir -p /content/Drive/
from google.colab import drive
drive.mount('/content/Drive') 
!mkdir -p /content/Drive/MyDrive/RetinaNet/
%cd /content/Drive/MyDrive/RetinaNet/

Mounted at /content/Drive
/content/Drive/MyDrive/RetinaNet


In [3]:
# !git clone https://github.com/fizyr/keras-retinanet
%cd /content/Drive/MyDrive/RetinaNet/keras-retinanet/
!pip install numpy --user
!pip install . --user
!python setup.py build_ext --inplace
# !wget https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5
# !mv  /content/Drive/MyDrive/RetinaNet/keras-retinanet/resnet50_coco_best_v2.1.0.h5  /content/Drive/MyDrive/RetinaNet//keras-retinanet/snapshots/


/content/Drive/MyDrive/RetinaNet/keras-retinanet
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/Drive/MyDrive/RetinaNet/keras-retinanet
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for keras-retinanet: filename=keras_retinanet-1.0.0-cp37-cp37m-linux_x86_64.whl size=169457 sha256=bd4aa5bb2e9af4db5d62c7e21fb99a5c9c2dcd71e5ddfee268df65a26275a60a
  Stored in directory: /root/.cache/pip/wheels/be/04/af/1c7daf7a6159b5274267d4629d60ed538b69299ec9996cc17f
  Created 

In [ ]:
%cd /content/Drive/MyDrive/RetinaNet/keras-retinanet/examples 
!python resnet50_retinanet.py

/content/Drive/MyDrive/RetinaNet/keras-retinanet/examples
1 Physical GPUs, 1 Logical GPUs
[[ 0  0  1  1  1  0  1  0  1  1  0  1  1  0  0  1  0  0  1  1  1  1  1 -1
  -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  -1 -1 -1 -1 -1 -1 -1

In [ ]:
%cd /content
!pip install roboflow

from roboflow import Roboflow
# rf = Roboflow(api_key="yGM3vBhGiZQUJrW3szSn")
# project = rf.workspace("tfm-pq95g").project("tfm-breast_640_separated")
# dataset = project.version(1).download("voc")

In [ ]:
!mkdir /content/Drive/MyDrive/RetinaNet/dataset
!mkdir /content/Drive/MyDrive/RetinaNet/dataset/{Annotations,ImageSets,JPEGImages}
!mkdir /content/Drive/MyDrive/RetinaNet/dataset/ImageSets/Main
!cp /content/TFM-breast_640_separated-1/test/*jpg /content/Drive/MyDrive/RetinaNet/dataset/JPEGImages/
!cp /content/TFM-breast_640_separated-1/train/*jpg /content/Drive/MyDrive/RetinaNet/dataset/JPEGImages/
!cp /content/TFM-breast_640_separated-1/valid/*jpg /content/Drive/MyDrive/RetinaNet/dataset/JPEGImages/
!cp /content/TFM-breast_640_separated-1/train/*xml /content/Drive/MyDrive/RetinaNet/dataset/Annotations/
!cp /content/TFM-breast_640_separated-1/test/*xml /content/Drive/MyDrive/RetinaNet/dataset/Annotations/
!cp /content/TFM-breast_640_separated-1/valid/*xml /content/Drive/MyDrive/RetinaNet/dataset/Annotations/

In [ ]:
from os import listdir
from os.path import isfile, join

# dataset_directory = "/content/dataset/Annotations"

with open("/content/Drive/MyDrive/RetinaNet/dataset/ImageSets/Main/test.txt", "w") as test:
    for file in listdir('/content/TFM-breast_640_separated-1/test/'):
        if ".xml" in file:
          continue
        name = file.split("\\")[-1]
        name = name.replace('.jpg', '')
        test.write(name + "\n")
        # print("test:    " + name)
        
with open("/content/Drive/MyDrive/RetinaNet/dataset/ImageSets/Main/train.txt", "w") as train:
    for file in listdir('/content/TFM-breast_640_separated-1/train/'):
        if ".xml" in file:
          continue
        name = file.split("\\")[-1]
        name = name.replace('.jpg', '')
        train.write(name + "\n")   
        # print("train:    " + name)   

with open("/content/Drive/MyDrive/RetinaNet/dataset/ImageSets/Main/val.txt", "w") as val:
    for file in listdir('/content/TFM-breast_640_separated-1/valid/'):
        if ".xml" in file:
          continue
        name = file.split("\\")[-1]
        name = name.replace('.jpg', '')
        val.write(name + "\n")   
        # print("train:    " + name)

test.close()
train.close()
val.close()

In [ ]:
# !cp -r /content/Drive/MyDrive/RetinaNet/dataset /content/

#empezar entrenamiento
# !python /content/Drive/MyDrive/RetinaNet/keras-retinanet/keras_retinanet/bin/train.py --imagenet-weights --lr 0.0001 --gpu 0 --epoch 100 --batch-size 8 --steps 614 --snapshot-path /content/Drive/MyDrive/RetinaNet/retinanet_models_2/ --tensorboard-dir /content/Drive/MyDrive/RetinaNet/retinanet_log_2/ pascal /content/Drive/MyDrive/RetinaNet/dataset/
#continuar entrenamiento
!python /content/Drive/MyDrive/RetinaNet/keras-retinanet/keras_retinanet/bin/train.py --initial-epoch 46 --snapshot /content/Drive/MyDrive/RetinaNet/retinanet_models_2/resnet50_pascal_45.h5 --lr 0.0001 --gpu 0 --epoch 100 --batch-size 8 --steps 614 --snapshot-path /content/Drive/MyDrive/RetinaNet/retinanet_models_2/ --tensorboard-dir /content/Drive/MyDrive/RetinaNet/retinanet_log_2/ pascal /content/dataset/

backbone
check_tf_version
1 Physical GPUs, 1 Logical GPUs
setup_gpu
read_config_file
PASCAL
PASCAL2
PASCAL3
create_generators
Loading model, this may take a second...
Model: "retinanet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1 (Conv2D)                 (None, None, None,   9408        ['input_1[0][0]']                
                                64)                                                               
                                                                                                  
 bn_conv1 (BatchNormal

In [ ]:
# generamos el modelo final
!python /content/Drive/MyDrive/RetinaNet/keras-retinanet/keras_retinanet/bin/convert_model.py /content/Drive/MyDrive/RetinaNet/retinanet_models/resnet50_pascal_49.h5 /content/Drive/MyDrive/RetinaNet/retinanet_models/resnet50_pascal_final.h5

In [ ]:
import sys
sys.path.insert(0, '/content/Drive/MyDrive/RetinaNet/keras-retinanet/keras_retinanet/')


# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

# use this to change which GPU to use
gpu = "0"

# set the modified tf session as backend in keras
setup_gpu(gpu)


# ## Load RetinaNet model

# In[ ]:


# adjust this to point to your downloaded/trained model
# models can be downloaded here: https://github.com/fizyr/keras-retinanet/releases
model_path = os.path.join('..', 'snapshots', '/content/Drive/MyDrive/RetinaNet/retinanet_models/resnet50_pascal_final.h5')

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')

# if the model is not converted to an inference model, use the line below
# see: https://github.com/fizyr/keras-retinanet#converting-a-training-model-to-inference-model
# model = models.convert_model(model)

#print(model.summary())

# load label to names mapping for visualization purposes
labels_to_names = {0: 'person', 1: 'bicycle', 2: 'car'}


# ## Run detection on example

# In[ ]:


# load image
image = read_image_bgr('/content/dataset/JPEGImages/Calc-Test_P_00038_LEFT_CC_png.rf.24a9e08dfd0318b78880e8dc2ee6fee1.jpg')

# copy to draw on
draw = image.copy()
draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

# preprocess image for network
image = preprocess_image(image)
image, scale = resize_image(image)

# process image
start = time.time()
boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
# print(boxes)
print(labels)
print("processing time: ", time.time() - start)

# correct for image scale
boxes /= scale

# visualize detections
for box, score, label in zip(boxes[0], scores[0], labels[0]):
    # scores are sorted so we can break
    if score < 0.5:
        break

    color = label_color(label)

    b = box.astype(int)
    draw_box(draw, b, color=color)

    caption = "{} {:.3f}".format(labels_to_names[label], score)
    draw_caption(draw, b, caption)

plt.figure(figsize=(15, 15))
plt.axis('off')
plt.imshow(draw)
plt.show()

In [ ]:
!cp /content/dataset/Annotations/Mass-Training_P_00294_LEFT_CC_png.rf.81f6012544c25a4afaf8242787c3ea44.xml /content/